<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN10/10.2-Verarbeitung_von_Sequenzen_mit_CNNs_pl.ipynb)

In Kapitel 5 haben Sie CNNs kennengelernt und erfahren, dass sie besonders gut
für Aufgabenstellungen des maschinellen Sehens geeignet sind. 
- Sie verdanken das ihrer Fähigkeit, durch Faltungsoperationen Merkmale **lokaler Eingabe-Patches** zu extrahieren. 
- Darüber hinaus ermöglichen sie **modulare Repräsentationen** und bieten eine effiziente Verwaltung der Daten. 
- Dieselben Eigenschaften, die dafür verantwortlich sind, dass sich CNNs beim maschinellen Sehen hervortun, sorgen auch dafür, dass ihnen bei der Verarbeitung von Sequenzen grosse Bedeutung zukommt. 
- **Die Zeit kann als eine räumliche Dimension behandelt** werden – wie die Höhe oder die Breite eines zweidimensionalen Bildes.

- Eindimensionale CNNs können sich bei bestimmten Aufgaben der Sequenzverarbeitung durchaus mit RNNs messen, benötigen dafür aber in der Regel einen **erheblich geringeren Rechenaufwand**. 
- In jüngster Zeit sind eindimensionale CNNs, die typischerweise einen erweiterten Kernel verwenden, mit großem Erfolg zur Klangerzeugung und zur maschinellen Übersetzung von Fremdsprachen eingesetzt worden.
- Von diesen gelungenen Ergebnissen einmal abgesehen, ist seit Langem bekannt, dass kleine eindimensionale CNNs bei einfachen Aufgaben wie Textklassifizierung und Vorhersage von Zeitreihen eine **schnelle Alternative zu RNNs** darstellen.

In [ ]:
# Imports
import re
import torch
import tarfile
import requests
import pandas as pd
import torch.nn as nn
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import torch.nn.functional as F

from pathlib import Path
from collections import Counter

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import DataLoader, TensorDataset


# 10.2 Eindimensionale Faltung sequenzieller Daten 
(sequence processing with convnets)

Dieses Notizbuch enthält die Codebeispiele aus Kapitel 6, Abschnitt 3 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). 


 Bei den bislang verwendeten Convolutional Layern handelte es sich um 2-D-Faltungen,
die Patches aus den Bildtensoren extrahieren und auf jeden Patch dieselbe
Transformation anwenden. Auf ähnliche Weise kann eine eindimensionale Faltung
verwendet werden, um lokale 1-D-Patches aus Sequenzen zu extrahieren
(also Untersequenzen, siehe folgende Abbildung).

<img src="Bilder/1D_CNN.png" width="440" align="center"/>


Diese eindimensionalen Convolutional Layer sind in der Lage, lokale Muster in
einer Sequenz zu erkennen. Da auf alle Patches dieselbe Transformation angewendet
wird, kann ein an einer bestimmten Stelle im Satz erlerntes Muster später an
einer anderen Stelle ebenfalls erkannt werden. 

- **Eindimensionale CNNs sind also translationsinvariant (bei zeitlichen Verschiebungen)**.
- Beispielsweise sollte ein eindimensionales CNN, das zur Verarbeitung einer Zeichensequenz ein Faltungsfenster der Grösse 5 verwendet, in einer Eingabesequenz 5 (oder weniger) Zeichen lange Wörter oder Wortfragmente erlernen können und in der Lage sein, diese Wörter in einem beliebigen Kontext wiederzuerkennen. 
- *Ein eindimensionales CNN zur Verarbeitung von Zeichen kann also die Form und Gestalt von Wörtern erlernen*.

### Eindimensionales Pooling sequenzieller Daten

Zweidimensionale Pooling-Operationen wie `Mean`- und `Max`-Pooling, die von CNNs zum räumlichen Downsampling von Bildtensoren verwendet werden, sind Ihnen bereits bekannt. Zu den zweidimensionalen Operationen gibt es eindimensionale Pendants:
- das Extrahieren von 1-D-Patches (Subsequenzen) aus einer Eingabe sowie Ausgabe des Maximalwerts (Max-Pooling) oder des Mittelwerts (Mean- oder Average-Pooling). Wie bei zweidimensionalen CNNs dienen diese Operationen dazu, die Länge der eindimensionalen Eingabe zu verringern (sogenanntes **Subsampling**).

## (a) Klassifikation mit einer rekurrenten 1D-CNN-Architektur Sentiment Analysis der imdb-Filmkritiken


In diesem Projekt führen wir eine Sentiment-Analyse auf dem bekannten IMDb-Filmrezensionsdatensatz durch. Ziel ist es, automatisch zu klassifizieren, ob eine gegebene Filmkritik positiv oder negativ ist. Dafür verarbeiten wir reinen Text mit Hilfe eines einfachen Tokenizers, erstellen ein manuelles Word-to-Index-Vokabular, und wandeln die Texte in numerische Sequenzen um. Diese Sequenzen werden anschließend auf eine einheitliche Länge gebracht (Padding), um sie in ein neuronales Netz einspeisen zu können.



In PyTorch können Sie für 1-D-CNNs den Conv1D-Layer einsetzen, der eine ähnliche Schnittstelle wie Conv2D besitzt. 
- Er nimmt einen **3-D-Tensor** mit der Shape `(samples, time, features)` als Eingabe entgegen und gibt Tensoren mit der gleichen Shape zurück. 
- Das Faltungsfenster ist ein 1-D-Fenster entlang der zeitlichen Achse: die Achse 1 des Eingabetensors.

Wir erzeugen nun ein einfaches zweischichtiges CNN und klassifizieren damit die
IMDb-Datensammlung. Zur Erinnerung hier der Code zum Einlesen und Vorbereiten
der Daten.

1. **Download & Extraktion** (nur wenn nötig)
2. **Laden der Daten** mit Labels & Tokenisierung
3. **Sequenzumwandlung** mittels Wortindex
4. **Padding der Sequenzen**
5. **Ausgabe der Shapes** der Trainings- und Testdaten

In [ ]:
# Hyperparameter
max_features = 10000
max_len = 500
data_dir = Path("data/aclImdb")


# Download & Extract IMDb Dataset
def download_and_extract_imdb(destination: Path):
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    dest_file = destination.parent / "aclImdb_v1.tar.gz"
    if not destination.exists():
        print("Downloading IMDb dataset...")
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(dest_file, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print("Extracting...")
        with tarfile.open(dest_file, "r:gz") as tar:
            tar.extractall(path=destination.parent)
        print("Done!")
    else:
        print("IMDb dataset already exists.")


# Tokenizer
def simple_tokenizer(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z']", " ", text)
    return text.split()


# IMDb Loader + Vocabulary
def load_imdb_data(path: Path, num_words: int):
    def load_texts_and_labels(subdir):
        texts, labels = [], []
        for label in ["pos", "neg"]:
            folder = path / subdir / label
            for file in sorted(folder.glob("*.txt")):
                texts.append(file.read_text(encoding="utf-8"))
                labels.append(1 if label == "pos" else 0)
        return texts, labels

    train_texts, train_labels = load_texts_and_labels("train")
    test_texts, test_labels = load_texts_and_labels("test")

    # Vokabular
    all_tokens = [token for text in train_texts for token in simple_tokenizer(text)]
    counter = Counter(all_tokens)
    most_common = counter.most_common(num_words - 2)
    word_index = {word: i + 2 for i, (word, _) in enumerate(most_common)}
    word_index["<PAD>"] = 0
    word_index["<UNK>"] = 1

    def texts_to_sequences(texts):
        sequences = []
        for text in texts:
            tokens = simple_tokenizer(text)
            indices = [word_index.get(token, 1) for token in tokens]  # 1 = <UNK>
            sequences.append(torch.tensor(indices, dtype=torch.long))
        return sequences

    x_train = texts_to_sequences(train_texts)
    x_test = texts_to_sequences(test_texts)
    y_train = torch.tensor(train_labels, dtype=torch.long)
    y_test = torch.tensor(test_labels, dtype=torch.long)

    return x_train, y_train, x_test, y_test, word_index


# Padding wie in Keras
def pad_sequences_torch(sequences, maxlen):
    padded = torch.zeros(len(sequences), maxlen, dtype=torch.long)
    for i, seq in enumerate(sequences):
        if len(seq) > maxlen:
            padded[i] = seq[:maxlen]
        else:
            padded[i, -len(seq) :] = seq  # rechtsbündig (post-padding)
    return padded


In [ ]:
download_and_extract_imdb(data_dir)

print("Loading data...")
x_train, y_train, x_test, y_test, word_index = load_imdb_data(data_dir, max_features)

print(len(x_train), "train sequences")
print(len(x_test), "test sequences")

print("Pad sequences (samples x time)")
x_train = pad_sequences_torch(x_train, max_len)
x_test = pad_sequences_torch(x_test, max_len)

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)


### IMDBTextCNN – PyTorch Lightning Modul für Textklassifikation
`IMDBTextCNN` ist ein einfaches, aber effektives Textklassifikationsmodell auf Basis eines 1D-CNN, implementiert mit PyTorch Lightning. Es nutzt eine Embedding-Schicht, eine 1D-Convolution, Adaptive Max-Pooling und eine vollverbundene Schicht mit Sigmoid-Ausgabe für binäre Sentiment-Analyse. Das Modell loggt Training- und Validierungsmetriken epochweise und verwendet den Adam-Optimizer.

In [ ]:
class IMDBTextCNN(pl.LightningModule):
    def __init__(
        self, vocab_size=10000, embed_dim=128, num_filters=64, kernel_size=5, lr=1e-3
    ):
        super().__init__()
        self.save_hyperparameters()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv = nn.Conv1d(embed_dim, num_filters, kernel_size)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(num_filters, 1)

        # Zwischenspeicher für Aggregation pro Epoche
        self.train_losses = []
        self.train_accs = []
        self.val_losses = []
        self.val_accs = []

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = F.relu(self.conv(x))
        x = self.pool(x).squeeze(-1)
        return torch.sigmoid(self.fc(x)).squeeze(-1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y.float())
        acc = ((y_hat > 0.5) == y).float().mean()
        self.train_losses.append(loss.detach())
        self.train_accs.append(acc.detach())
        return loss

    def on_train_epoch_end(self):
        mean_loss = torch.stack(self.train_losses).mean()
        mean_acc = torch.stack(self.train_accs).mean()
        self.log("train_loss", mean_loss, prog_bar=True, on_epoch=True)
        self.log("train_acc", mean_acc, prog_bar=True, on_epoch=True)
        self.train_losses.clear()
        self.train_accs.clear()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.binary_cross_entropy(y_hat, y.float())
        acc = ((y_hat > 0.5) == y).float().mean()
        self.val_losses.append(loss.detach())
        self.val_accs.append(acc.detach())

    def on_validation_epoch_end(self):
        mean_loss = torch.stack(self.val_losses).mean()
        mean_acc = torch.stack(self.val_accs).mean()
        self.log("val_loss", mean_loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", mean_acc, prog_bar=True, on_epoch=True)
        self.val_losses.clear()
        self.val_accs.clear()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)


`IMDBDataModule` ist ein PyTorch Lightning DataModule, das Trainings- und Validierungsdaten kapselt. Es wandelt Eingabedaten (`x_train`, `y_train`, `x_val`, `y_val`) in `TensorDataset`s um und stellt über `train_dataloader()` und `val_dataloader()` die entsprechenden DataLoader bereit – inklusive Batch-Größe und Shuffling für das Training.

In [ ]:
class IMDBDataModule(pl.LightningDataModule):
    def __init__(self, x_train, y_train, x_val, y_val, batch_size=128):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_ds = TensorDataset(self.x_train, self.y_train)
        self.val_ds = TensorDataset(self.x_val, self.y_val)

    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size)


Hier wird das Training des `IMDBTextCNN`-Modells mit PyTorch Lightning gestartet:

- `IMDBDataModule` bereitet die Trainings- und Testdaten vor.
- `IMDBTextCNN` ist das zu trainierende Textklassifikationsmodell.
- `CSVLogger` speichert Metriken wie Loss und Accuracy in CSV-Dateien unter `logs/imdb_textcnn/`.
- `Trainer` führt das Training über 15 Epochen aus, wählt automatisch das passende Gerät (CPU/GPU).
- `trainer.fit(...)` startet das eigentliche Training mit dem DataModule und dem Modell.

In [ ]:
data_module = IMDBDataModule(x_train, y_train, x_test, y_test, batch_size=128)
model = IMDBTextCNN(vocab_size=max_features)

csv_logger = CSVLogger("logs", name="imdb_textcnn")

trainer = pl.Trainer(
    max_epochs=15,
    logger=csv_logger,
    accelerator="auto",
)

trainer.fit(model, data_module)


Dieser Code lädt die Trainings-Logs des Modells aus der `metrics.csv`-Datei (erstellt vom `CSVLogger`) und visualisiert den **Verlauf des Trainings- und Validierungsverlusts (Loss) über die Epochen**:

### Schritte im Überblick:

1. **CSV-Datei laden**  
   ```python
   df = pd.read_csv(metrics_path)
   ```
   - Liest die während des Trainings gespeicherten Metriken ein.

2. **Trainings- und Validierungsdaten filtern**  
   ```python
   train_df = df[df["train_loss"].notna()]
   val_df = df[df["val_loss"].notna()]
   ```
   - Trennt Zeilen, in denen `train_loss` bzw. `val_loss` vorhanden sind.

3. **Loss-Kurven plotten**  
   ```python
   plt.plot(...)
   ```
   - Zeichnet zwei Linienplots: eine für `train_loss`, eine für `val_loss`.
   - Marker (`o` und `x`) sowie unterschiedliche Linienstile sorgen für bessere Lesbarkeit.
   - Achsen, Titel, Legende und Grid werden ergänzt für eine saubere Visualisierung.

### Ergebnis:
Ein Plot, der zeigt, wie sich der Loss auf Trainings- und Validierungsdaten über die Epochen hinweg entwickelt – hilfreich zur Beurteilung von **Lernfortschritt**, **Overfitting** oder **Underfitting**.

In [ ]:
metrics_path = csv_logger.log_dir + "/metrics.csv"
df = pd.read_csv(metrics_path)

# Zeilen mit train_loss und val_loss extrahieren
train_df = df[df["train_loss"].notna()]
val_df = df[df["val_loss"].notna()]

# Plotten
plt.plot(
    train_df["epoch"],
    train_df["train_loss"],
    label="Train Loss",
    marker="o",
    linestyle="-",
)
plt.plot(
    val_df["epoch"], val_df["val_loss"], label="Val Loss", marker="x", linestyle="--"
)
plt.title("Loss Curve (1D-CNN on IMDb)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.show()


## (b) Regression mit rekurrenten 1D-CNN-Modellen: Zeitreihen-Vorhersage der Jena Wetterdaten

In diesem Abschnitt wenden wir 1D-Convolutional Neural Networks (1D-CNNs) auf die Jena-Klimadaten an, um zukünftige Wetterwerte auf Basis historischer Messungen vorherzusagen. Die Idee dabei ist, zeitliche Muster in den Daten zu erkennen, ohne auf rekurrente Strukturen wie RNNs oder LSTMs zurückzugreifen.

Ein 1D-CNN funktioniert hier als Feature-Extractor, der lokal zeitliche Abhängigkeiten in den Input-Sequenzen identifiziert – ähnlich wie ein CNN in der Bildverarbeitung lokale Muster erkennt. Durch den Einsatz mehrerer Faltungsschichten können auch komplexere Zusammenhänge über längere Zeitspannen modelliert werden.


Als nächstes wird:
1. CSV-Datei eingelesen und
2. normalisiert (Standardisierung)
### Ziel:
- Sicherstellen, dass alle Eingabefeatures **vergleichbare Skalen** haben.
- Verbessert Stabilität und Lernverhalten von neuronalen Netzen.

In [ ]:
# CSV einlesen
df = pd.read_csv("data/jena_climate_2009_2016.csv")
float_data = df.iloc[:, 1:].values  # alle außer Zeitstempel

# Normierung (wie in Keras)
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std


`TimeseriesDataset` ist eine benutzerdefinierte PyTorch-Dataset-Klasse zur Vorbereitung von **Zeitreihendaten für Vorhersageaufgaben** (z. B. Temperaturprognose auf Basis vergangener Messwerte).

### Was passiert?

- Nutzt ein gleitendes Fenster über normalisierte Klimadaten.
- Für jeden Datenpunkt wird ein **Input-Window (`lookback`)** erstellt.
- Zielwert (`target`) ist die **Temperatur (`data[:, 1]`) nach `delay` Zeitschritten**.

### Parameter:

- `lookback`: Wie viele Zeitschritte in die Vergangenheit als Input dienen.
- `delay`: Wie viele Zeitschritte in die Zukunft vorhergesagt werden sollen.
- `step`: Intervall zwischen zwei Zeitpunkten im Input-Window.
- `min_index`, `max_index`: Begrenzung des Datenbereichs (z. B. für Training/Validierung/Test).

### Rückgabe:

- Ein `sample`: Input-Zeitfenster (mehrere Merkmale über Zeit)
- Ein `target`: Temperaturwert in der Zukunft

Verwendbar mit einem `DataLoader` für effizientes Batch-Training.

In [ ]:
class TimeseriesDataset(Dataset):
    def __init__(self, data, lookback, delay, min_index, max_index, step=6):
        self.data = data
        self.lookback = lookback
        self.delay = delay
        self.step = step
        self.min_index = min_index + lookback
        self.max_index = max_index if max_index is not None else len(data) - delay - 1

    def __len__(self):
        return self.max_index - self.min_index

    def __getitem__(self, idx):
        i = self.min_index + idx
        indices = range(i - self.lookback, i, self.step)
        sample = self.data[indices]
        target = self.data[i + self.delay][1]  # Temperatur (2. Spalte)
        return torch.tensor(sample, dtype=torch.float32), torch.tensor(
            target, dtype=torch.float32
        )


`JenaDataModule` ist ein **PyTorch Lightning DataModule** zur Organisation der Jena-Klimadaten für Zeitreihenmodelle.

### Hauptfunktionen:

- Teilt die normalisierten Daten (`float_data`) in:
  - **Training** (0 – 200.000)
  - **Validierung** (200.001 – 300.000)
  - **Test** (ab 300.001)
- Verwendet dabei das `TimeseriesDataset`, das Eingabefenster (`lookback`) und Zielwerte (`delay`) für Vorhersagen erstellt.

### Parameter:

- `lookback`: Anzahl vergangener Zeitschritte als Input
- `delay`: Zielzeitpunkt in der Zukunft
- `step`: Zeitintervall innerhalb des Inputfensters
- `batch_size`: Größe der Trainings- und Test-Batches

### Methoden:

- `train_dataloader()`, `val_dataloader()`, `test_dataloader()` liefern passende DataLoader für den Trainingsprozess.

Dieses Modul kapselt die gesamte **zeitliche Datenaufbereitung und Batch-Erstellung** – sauber und wiederverwendbar für Lightning-Modelle.

In [ ]:
class JenaDataModule(pl.LightningDataModule):
    def __init__(self, float_data, lookback=1440, delay=144, step=6, batch_size=128):
        super().__init__()
        self.float_data = float_data
        self.lookback = lookback
        self.delay = delay
        self.step = step
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train = TimeseriesDataset(
            self.float_data, self.lookback, self.delay, 0, 200000, self.step
        )
        self.val = TimeseriesDataset(
            self.float_data, self.lookback, self.delay, 200001, 300000, self.step
        )
        self.test = TimeseriesDataset(
            self.float_data, self.lookback, self.delay, 300001, None, self.step
        )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batch_size)


`JenaCNN` ist ein Modell zur Vorhersage zukünftiger Temperaturen auf Basis historischer Klimadaten. Es verwendet ein eindimensionales Convolutional Neural Network (1D-CNN), um zeitliche Muster in den Daten zu erkennen. Die Architektur besteht aus mehreren Faltungs- und Pooling-Schritten zur Merkmalextraktion und einer finalen Regressionsausgabe. Als Fehlerfunktion wird der mittlere absolute Fehler (MAE) verwendet. Das Modell nutzt RMSprop zur Optimierung.

In [ ]:
class JenaCNN(pl.LightningModule):
    def __init__(self, num_features, lr=1e-4):
        super().__init__()
        self.save_hyperparameters()

        self.model = nn.Sequential(
            nn.Conv1d(num_features, 32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool1d(3),
            nn.Conv1d(32, 32, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool1d(3),
            nn.Conv1d(32, 32, kernel_size=5),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
        )
        self.fc = nn.Linear(32, 1)
        self.loss_fn = nn.L1Loss()  # MAE = Mean Absolute Error

    def forward(self, x):
        x = x.permute(0, 2, 1)  # (B, T, F) → (B, F, T)
        x = self.model(x).squeeze(-1)  # (B, C, 1) → (B, C)
        return self.fc(x).squeeze(-1)  # (B)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=self.hparams.lr)


In diesem Abschnitt wird das 1D-CNN-Modell `JenaCNN` mit dem vorbereiteten `JenaDataModule` trainiert. Das DataModule organisiert die Zeitreihendaten in Trainings-, Validierungs- und Testsets. Das Modell wird mit einem PyTorch Lightning `Trainer` für 10 Epochen trainiert, wobei alle Metriken über einen CSV-Logger gespeichert werden. Die Hardware (CPU/GPU) wird automatisch gewählt.

In [ ]:
# DataModule
data = JenaDataModule(float_data)
data.setup()

# Modell
num_features = float_data.shape[-1]
model = JenaCNN(num_features=num_features)

# Trainer
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

trainer = Trainer(
    max_epochs=10, logger=CSVLogger("logs", name="jena-cnn"), accelerator="auto"
)

trainer.fit(model, datamodule=data)


Auch hier werden wieder die Kurven geplottet...

In [ ]:
metrics_path = trainer.logger.log_dir + "/metrics.csv"
df = pd.read_csv(metrics_path)
# Zeilen mit train_loss und val_loss extrahieren
train_df = df[df["train_loss"].notna()]
val_df = df[df["val_loss"].notna()]
# Plotten
plt.plot(
    train_df["epoch"],
    train_df["train_loss"],
    label="Train Loss",
    marker="o",
    linestyle="-",
)
plt.plot(
    val_df["epoch"], val_df["val_loss"], label="Val Loss", marker="x", linestyle="--"
)
plt.title("Loss Curve (1D-CNN on Jena)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()
plt.show()


`JenaCNNGRU` ist ein hybrides Modell zur Zeitreihenvorhersage, das **1D-CNNs** und eine **GRU** kombiniert. Die CNN-Schichten extrahieren lokale Muster aus den Eingabedaten, während die GRU die zeitlichen Abhängigkeiten verarbeitet. Das Modell gibt am Ende einen einzelnen Regressionswert (z. B. zukünftige Temperatur) aus. Es verwendet MAE als Loss und wird mit RMSprop optimiert. 

In [ ]:
class JenaCNNGRU(pl.LightningModule):
    def __init__(self, num_features, lr=1e-4):
        super().__init__()
        self.save_hyperparameters()

        # Feature-Extraktion (lokale Muster)
        self.conv1 = nn.Conv1d(num_features, 32, kernel_size=5)
        self.pool1 = nn.MaxPool1d(kernel_size=3)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5)

        # Zeitliches Kontextverständnis
        self.gru = nn.GRU(
            input_size=32,
            hidden_size=32,
            batch_first=True,
            dropout=0.5,  # fester Wert wie in Keras
        )

        # Regressionsoutput
        self.fc = nn.Linear(32, 1)

        # MAE = Mean Absolute Error
        self.loss_fn = nn.L1Loss()

    def forward(self, x):
        # Eingabe: (B, T, F)
        x = x.permute(0, 2, 1)  # (B, F, T)
        x = torch.relu(self.conv1(x))  # (B, 32, T')
        x = self.pool1(x)  # (B, 32, T'')
        x = torch.relu(self.conv2(x))  # (B, 32, T''')

        x = x.permute(0, 2, 1)  # (B, T, C) für GRU
        out, _ = self.gru(x)  # (B, T, H)
        last_hidden = out[:, -1, :]  # nur letztes Zeitschritt
        return self.fc(last_hidden).squeeze(-1)  # (B,)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=self.hparams.lr)


Hier wird das hybride Modell `JenaCNNGRU`, das CNNs und GRUs kombiniert, für 10 Epochen mit dem `JenaDataModule` trainiert. Der `Trainer` von PyTorch Lightning übernimmt das Training, speichert Metriken mit einem CSV-Logger und wählt automatisch die passende Hardware (CPU oder GPU).

In [ ]:
model = JenaCNNGRU(num_features=float_data.shape[-1])
trainer = pl.Trainer(
    max_epochs=10, logger=CSVLogger("logs", name="jena-cnn-gru"), accelerator="auto"
)
trainer.fit(model, datamodule=data)


In [ ]:
metrics_path = trainer.logger.log_dir + "/metrics.csv"
df = pd.read_csv(metrics_path)
# Filtere die Zeilen, in denen ein Wert für train_loss bzw. val_loss steht
train = df[df["train_loss"].notna()]
val = df[df["val_loss"].notna()]


# Plotten
plt.figure(figsize=(8, 5))
plt.plot(
    train["epoch"], train["train_loss"], label="Train Loss", marker="o", linestyle="-"
)
plt.plot(val["epoch"], val["val_loss"], label="Val Loss", marker="x", linestyle="--")
plt.title("Loss-Kurve: CNN + GRU (Jena-Klimadaten)")
plt.xlabel("Epoch")
plt.ylabel("Loss (MAE)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


## Where's the intelligence?

- In welchen Situationen zeigen RNNs bessere Performance als 1D-CONV-Netze? 
- Wie könnte man CONV-Netze dahingehend verbessern?
- Wie wird bei RNN das *vanishing gradients* Problem gelöst?
